In [ ]:
# Optimisation Algorithms in Non-Convex Classification Problems with Adversarial Training

In [ ]:
# MINI-BATCH GRADIENT

In [ ]:
# Import data from MNIST

In [ ]:
import torch
use_cuda = False # GPU seems to raise erros on my side
device = torch.device('cuda' if use_cuda and torch.cuda.is_available() else 'cpu')
from data_utils import get_mnist, build_data_loaders
train_dataset, test_dataset = get_mnist(normalize=True)
train_dataset.data.mean(0).mean().item(), test_dataset.data.mean(0).mean().item()

In [ ]:

# Values should be (-7.328089801639237e-10, 0.002495675580576062) if the import was successful

In [ ]:
# Tuning Hyperparameters for Minibatch to ensure equitable comparision to the other algorithms tested
# Use the Ray Tune library to run Bayes Search on the following parameters:
# The learning rate
# The batch size
# Whether we have a decreasing learning rate or not

In [ ]:
from ray import tune
from net import Net
import numpy as np 
import pandas
from minibatch import minibatch_run, minibatch_tune

In [ ]:
from ray.tune.suggest.bayesopt import BayesOptSearch
space = {
            "lr": tune.uniform(1e-4, 1e-2),
            "b_size": tune.uniform(2, 128),
            "dec_lr": tune.uniform(0, 1)
        }
samples = 5 # Suggested value by the Ray Tune Docs

In [ ]:
# Note here that the neural net deployed here is found in the net.py file
# The Loss function throughout is simply Torch's CrossEntropyLoss
# Bayes Search here focuses on maximising the average accuracy (out of 100%)

In [ ]:
tune_net = Net().to(device)
criterion = torch.nn.CrossEntropyLoss()
bayesopt = BayesOptSearch(metric="mean_accuracy", mode="max")

In [ ]:
# The tuning function is found in the minibatch.py file and based on the standard format for Ray Tune calls
# It stores the result of the 10 runs in a pandas dataframe and then returns the highest average accuracy and the associated parameters

In [ ]:
best_config, best_result = minibatch_tune(tune_net, train_dataset, test_dataset, samples, criterion, space, bayesopt, device)

In [ ]:
best_config

In [ ]:
epochs = 10
batch_size = 91
learning_rate = 0.01
decreasing_lr = False

In [ ]:
epochs = 10
batch_size = int(best_config["b_size"])
learning_rate = best_config["lr"]
decreasing_lr = best_config["dec_lr"]

In [ ]:
if decreasing_lr < 0.5:
    print("Best Result was {:.2f}% with a Batch Size of {} and a fixed learning rate of {:.4f}".format(best_result*100, batch_size, learning_rate))
else:
    print("Best Result was {:.2f}% with a Batch Size of {} and a learning rate of {:.4f} that was decreasing".format(best_result*100, batch_size, learning_rate))

In [ ]:
# Training and Testing under Normal Conditions

In [ ]:
net_naive = Net().to(device)
train_loader, test_loader = build_data_loaders(train_dataset, test_dataset, batch_size)
for x, y in train_loader:
    print(x.shape, y.shape)
    break

In [ ]:
### Test the setup with the Minibatch Optimiser from the minibatch.py file
### Accuracy here is the percentage of correctly guessed labels

In [ ]:
from training import training, testing, accuracy
from minibatch import MiniBatchOptimizer

mini_opt = MiniBatchOptimizer(net_naive.parameters(), lr=learning_rate, decreasing_lr=decreasing_lr)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
losses, acc = training(net_naive, train_loader, mini_opt, criterion, accuracy, epochs=epochs, device=device)

In [ ]:
losses, acc = testing(net_naive, test_loader, criterion, accuracy, device=device)
plt.plot(losses)
plt.show()
plt.plot(acc)
plt.show()

In [ ]:
#net_compact = Net().to(device)

In [ ]:
# losses, accs = minibatch_run(net_compact, train_dataset, test_dataset, criterion, accuracy, device=device, bz=batch_size, lr=learning_rate, epochs=epochs, dec_lr=decreasing_lr)
# Has the weird shaping issue

In [ ]:
## Plot of losses

In [ ]:
plt.plot(losses)
plt.show()
plt.plot(acc)
plt.show()

In [ ]:
# Attack against Unprotected Model

In [ ]:
from adversary import attack, protect

In [ ]:
accuracies_lenet= []
examples_lenet = []

epsilons_lenet = np.arange(0, 0.5, 0.05)

In [ ]:
epsilons_lenet

In [ ]:
### Attack the model with different epsilons

In [ ]:
for eps in epsilons_lenet:
    acc, ex = attack(net_naive, criterion, test_loader, update_max_norm=eps, device=device)
    accuracies_lenet.append(acc)
    examples_lenet.append(ex)

In [ ]:
### Plot Results of Attacks

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons_lenet, accuracies_lenet, "*-")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
# Protect Model against FGSM

In [ ]:
robust_net = Net().to(device)
protect_epochs = 10
protect_lr = 0.01
protect_bz = 16
protect_dec_lr = False

In [ ]:
prot_train_loader, prot_test_loader = build_data_loaders(train_dataset, test_dataset, protect_bz)
mini_opt = MiniBatchOptimizer(robust_net.parameters(), lr=protect_lr, decreasing_lr=protect_dec_lr)

In [ ]:
#(model: Module, optim: Optimizer,criterion: Module, train_loader: Iterable, test_loader: Iterable, device, epochs: int = 10):
robust_net = protect(robust_net, mini_opt, criterion, prot_train_loader, prot_test_loader, device=device, epochs=protect_epochs)

In [ ]:
robust_net

In [ ]:
for epoch in range(protect_epochs):
        # Train an epoch
        #robust_net.train()
        for batch_x, batch_y in prot_train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Forward pass for adversarial perturbations
            batch_x.requires_grad = True
            output = robust_net(batch_x)

            original_predictions = output.argmax(1) # get the index of the max logit
            original_accuracy = accuracy(output, batch_y)
            loss = criterion(output, batch_y)
            robust_net.zero_grad()
            loss.backward()
            perturbed_data = fgsm(batch_x, batch_x.grad, 0.25)
            
            # Evaluate the network (forward pass)
            prediction = robust_net(perturbed_data)
            loss = criterion(prediction, batch_y)
            
            # Compute the gradient
            mini_opt.zero_grad()
            loss.backward()

            # Update the parameters of the model with a gradient step
            mini_opt.step()

        # Test the quality on the test set
        robust_net.eval()
        accuracies = []
        for batch_x, batch_y in prot_test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Evaluate the network (forward pass)
            prediction = robust_net(batch_x)
            accuracies.append(accuracy(prediction, batch_y))
        
        print("Epoch {:.2f} | Test accuracy: {:.5f}".format(epoch, sum(accuracies).item()/len(accuracies)))    

In [ ]:
# Attack against Protected Model

In [ ]:
accuracies_lenet_robust = []
examples_lenet_robust = []

epsilons_lenet_robust = np.arange(0, 0.5, 0.05)

In [ ]:
for eps in epsilons_lenet_robust:
    acc, ex = attack(protected_net, CrossEntropyLoss(), prot_train_loader, eps, device=device)
    accuracies_lenet_robust.append(acc)
    examples_lenet_robust.append(ex)

In [ ]:
# Comparing the models
plt.figure(figsize=(5,5))
plt.plot(epsilons_lenet, accuracies_lenet, "*-", c='blue', label='Convolutional network')
plt.plot(epsilons_lenet_robust, accuracies_lenet_robust, "*-", c='orange', label='Convolutional network (robust)')

plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))

plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.legend();